# Data Mining Assignment
KLSE Glove Company Stock Price Prediction

Name: Low Tsu Siang

Matrix Number: WQD180072

### Background Story
The outcome of this assignment is to develop a web app with the function of the stock price prediction. History data of the stock data was collected via web scrapping from respective website. The collected data was managed & then stored in data lake. Microsoft Azure's blob container was choosen as the data lake platform for this project. 
Regresion model was selected as prediction model for this application. Data can be easily restored from data lake & run the necessary processes when internet connection is available. No security limitation was set for the data lake database. 
Flask web app is developed as the UI for this prediction model. 


### Milestone 1: Web scraping
Web scarping stock information from website

In [ ]:
sector_key_value = [{"main_market": "healthcare"}]
sector_key_value = [{"main_market": "healthcare"}]
stock_list = []
for item in sector_key_value:
    for k, v in item.items():
        stock_list.append(["https://s3-ap-southeast-1.amazonaws.com/biz.thestar.com.my/json/sectors/{0}/{1}/stocks.js".
                          format(k, v), k, v])

In [ ]:
## Collect the data of selected counter

def company_name(js_urls):
    for url in js_urls:
#        try:
            res = urllib.request.urlopen(url[0])
            body = str(res.read()).split("=")
            body = body[1].replace("\\r\\n", "")
            body = body.replace(body[0:14], "")
            body = body.replace("]};\'", "")
            data = eval(body)
            try:
                for item in data:
                    crawl_data(item, url[1], url[2])
            except:
                crawl_data(data, url[1], url[2])
#        except:
#            print("error_stop1")
#            continue


def crawl_data(name, sector, subsector):
    start_date = "1230872400"
    end_date = "1577826000"
    url = "https://charts.thestar.com.my/datafeed-udf/history?symbol={0}&resolution=D&from={1}&to={2}".format(name["counter"], start_date, end_date)
    res = urllib.request.urlopen(url)
    body = res.read().decode()
    dict_data = eval(body)
#    print(dict_data)
    df=pd.DataFrame(dict_data)
    path="C:\\Users\\ts\\Documents\\Data Mining\\Data\\"
    #print(path)
    csv_file = format(name["counter"])+".csv"
    #print(csv_file)
    df.to_csv (os.path.join(path,csv_file), index = False, header=True)

if __name__ == "__main__":
    company_name(stock_list)

### Milestone 2: Store data at Data Lake 
Pre-requisite: 
1. Create a Microsoft Azure account (student account is available for free)
2. Create Block Blob Service

In [ ]:
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess
block_blob_service = BlockBlobService(account_name='wqd7005', account_key='XpndfNFVKvJKGU22QzDYnw8VKc91G4QIW3JESEZQSO4hxLkCB16O+F0wdo6IAAuPvJ+FizhoDO7AWKm4x9N6xw==')

In [ ]:
# Create a container called 'rawblob'.
container_name ='rawblob'
block_blob_service.create_container(container_name)
# Set the permission so the blobs are public.
block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

In [ ]:
# Upload the created file, use local_file_name for the blob name
import os
local_path=os.path.abspath(os.path.curdir)
path=os.path.join(local_path, "Data")

def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

for file in files(os.path.join(local_path, "Data")):
    local_file_name=os.path.join("Data",file)
    full_path_to_file =os.path.join(local_path, local_file_name)
    block_blob_service.create_blob_from_path(container_name, local_file_name, full_path_to_file)
    print (file)

In [ ]:
#Check available data
print("\nList blobs in the container")
generator = block_blob_service.list_blobs(container_name)
for blob in generator:
    print("\t Blob name: " + blob.name)

2 ways to retrieve data from Data Lake:
1. Download as csv & store in harddrive
2. Download as dictionary within python platform

In [ ]:
# Download the blob(s) to local drive in csv format
# Add '_DOWNLOADED' as prefix to '.txt' so you can see both files in Documents.
generator = block_blob_service.list_blobs(container_name)
for blob in generator:
#    print("\t Blob name: " + blob.name)
    local_file_name=os.path.join(blob.name)
    full_path_to_file2 = os.path.join(local_path, str.replace(local_file_name ,'.csv', '_DOWNLOADED.csv'))
    print("\nDownloading blob to " + full_path_to_file2)
    block_blob_service.get_blob_to_path(container_name, blob.name, full_path_to_file2)
    

    
    
## Download blob in dataframe
from io import StringIO
import pandas as pd
generator = block_blob_service.list_blobs(container_name)
tables = {}
for blob in generator:
    local_file_name=os.path.join(blob.name)
    #from azure.storage.blob import BlobService
    #print(local_file_name)
    blobstring = block_blob_service.get_blob_to_text(container_name,local_file_name).content
    #df = pd.read_csv(StringIO(blobstring))
    names1 = blob.name[5:]
    names = names1[:-4]
    print(names)
    tables[names] = pd.read_csv(StringIO(blobstring))    


In [1]:
from urllib.request import Request
import urllib
from datetime import datetime
import numpy as np
import pandas as pd
import csv
import os
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import plotly.offline as po
import plotly.graph_objs as go

Retrive data from Data lake & store in dictionary as the stock counter name

In [2]:
##Pre-defined
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess
block_blob_service = BlockBlobService(account_name='wqd7005', account_key='XpndfNFVKvJKGU22QzDYnw8VKc91G4QIW3JESEZQSO4hxLkCB16O+F0wdo6IAAuPvJ+FizhoDO7AWKm4x9N6xw==')
container_name ='rawblob'

## Download blob in dataframe
from io import StringIO
import pandas as pd
generator = block_blob_service.list_blobs(container_name)
tables = {}
for blob in generator:
    local_file_name=os.path.join(blob.name)
    #from azure.storage.blob import BlobService
    #print(local_file_name)
    blobstring = block_blob_service.get_blob_to_text(container_name,local_file_name).content
    #df = pd.read_csv(StringIO(blobstring))
    names1 = blob.name[5:]
    names = names1[:-4]
    print(names)
    tables[names] = pd.read_csv(StringIO(blobstring))

ADVENTA
AHEALTH
DPHARMA
HARTA
IHH
KOSSAN
KOTRA
KPJ
PHARMA
SUPERMX
TMCLIFE
TOPGLOV
YSPSAH


Data Transformation: Convert time format

In [3]:
for name in list(tables):
    tables[name].iloc[:,0] = pd.to_datetime(tables[name].iloc[:,0], unit='s').dt.strftime('%Y-%m-%d')
    #print(name)
    #print(tables[name].iloc[:,0])

## Use counter KOSSAN as sample

In [4]:
df=tables['KOSSAN']

In [5]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas_datareader.data as web
style.use('ggplot')
df=tables['KOSSAN']
df['100ma']=df['c'].rolling(window=100,min_periods=0).mean()
df['20ma']=df['c'].rolling(window=20,min_periods=0).mean()
df.dropna()

D:\software\anaconda\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



,t,c,o,h,l,v,s,100ma,20ma
0,2009-01-01,0.723,0.698,0.723,0.698,357600,ok,0.723000,0.723000
1,2009-01-04,0.753,0.726,0.753,0.726,118300,ok,0.738000,0.738000
2,2009-01-05,0.748,0.758,0.783,0.743,430500,ok,0.741333,0.741333
3,2009-01-06,0.768,0.758,0.778,0.758,160200,ok,0.748000,0.748000
4,2009-01-07,0.763,0.763,0.763,0.746,122400,ok,0.751000,0.751000
...,...,...,...,...,...,...,...,...,...
2691,2019-12-23,4.170,4.180,4.180,4.160,45900,ok,4.188300,4.153000
2692,2019-12-25,4.200,4.160,4.250,4.160,312700,ok,4.189600,4.153000
2693,2019-12-26,4.250,4.180,4.250,4.160,311200,ok,4.191400,4.155500
2694,2019-12-29,4.200,4.240,4.240,4.190,467800,ok,4.192700,4.156000


In [6]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
get_ipython().magic('matplotlib qt')
ax1=plt.subplot2grid((13,2),(0,0),rowspan=10, colspan=2)
ax2=plt.subplot2grid((13,2),(11,0),rowspan=2, colspan=2, sharex=ax1)

ax1.plot(df['t'], df['c'], label="Closing Price")
ax1.plot(df['100ma'], label="100ma")
ax1.plot(df['20ma'], label="20ma")
ax1.legend(loc='upper left', frameon=False)

ax2.bar(df.index, df['v'])
ax2.set_ylabel('Volume', fontsize=8)

#set ticks every week
ax1.xaxis.set_major_locator(mdates.MonthLocator())
#set major ticks format
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))



plt.savefig('foo.png')

## Using Regression - testing purpose before deploy in web app

In [10]:
#For Prediction
## Regression
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
%matplotlib inline

In [12]:
X = df[['o','h','l','v']]
y = df['c']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regressor = LinearRegression()  
regressor.fit(X_train, y_train)
#coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])  
#coeff_df

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
o,-4.633118e-01
h,7.614298e-01
l,7.024062e-01
v,-3.727089e-09


In [14]:
y_pred = regressor.predict(X_test)

In [15]:
result = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
result

,Actual,Predicted
195,1.220,1.225610
2342,8.470,8.500168
1597,6.530,6.595419
2537,3.680,3.643404
1457,4.350,4.391626
...,...,...
914,1.502,1.500108
357,1.847,1.844194
910,1.507,1.512508
682,1.437,1.471588


In [16]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.021540659196480683
Mean Squared Error: 0.001140308335129862
Root Mean Squared Error: 0.033768451772769534


### Function for Web App 

In [17]:
def prediction(symbol):

    df = tables[symbol]
    X = df[['o','h','l','v']]
    y = df['c']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = LinearRegression()  
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X)
    result = pd.DataFrame({'Actual': y, 'Predicted': y_pred})
    return result

In [18]:
def get_graph(symbol):
    data = prediction(symbol)
    SMA5  = data['Actual'].rolling(5).mean()
    Regression = data['Predicted']
    Close = data['Actual']
    s5 = go.Scatter(x = SMA5.index,y = SMA5.values,name = '5MA')
    cls = go.Scatter(x = Close.index,y = Close.values,name = 'Close')
    reg = go.Scatter(x = Regression.index,y = Regression.values,name = 'Regression')
    data = [s5,cls,reg]
    layout = {'title': symbol}
    fig = dict(data=data, layout=layout)
    po.plot(fig, filename='templates/stock.html',auto_open=False)
    return True



In [19]:
def result(symbol,num1, num2, num3, num4):
    df = tables[symbol]
    X = df[['o','h','l','v']]
    y = df['c']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = LinearRegression()  
    regressor.fit(X_train, y_train)
    Xnum1 = float(num1)
    Xnum2 = float(num2)
    Xnum3 = float(num3)
    Xnum4 = int(num4)
    X_result=[]
    X_result = [(Xnum1, Xnum2, Xnum3, Xnum4)]
    result = regressor.predict(X_result)
    return result


### Web App main application 

In [ ]:
from pandas_datareader import data as web
from io import BytesIO
from flask import Flask, request, render_template, redirect, url_for
#from forms import SymbolSearch
import base64
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.dates import DateFormatter

app = Flask(__name__)
symbol2={}

@app.route('/')
def home():
    return render_template('index.html')

@app.route("/index", methods=['GET', 'POST'])
def index():
    error = None
    if request.method == 'POST':
        symbol=request.form['symbol']
        result = get_graph(symbol)
        if result == True:
            return render_template("stock.html")
        else:
            return render_template('selection.html', sign='notfound')
    else:
        return render_template('selection.html')

@app.route('/send', methods=['POST'])
def send(sum=sum):
    if request.method == 'POST':
        num1 = request.form['num1']
        num2 = request.form['num2']
        num3 = request.form['num3']
        num4 = request.form['num4']
        
        operation = request.form['operation']

        if operation == 'KOSSAN':
            sum = result(operation, num1, num2, num3, num4)
            return render_template('index.html', sum=sum)

        elif operation == 'HARTA':
            sum = result(operation, num1, num2, num3, num4)
            return render_template('index.html', sum=sum)

        elif operation == 'TOPGLOV':
            sum = result(operation, num1, num2, num3, num4)
            return render_template('index.html', sum=sum)

        elif operation == 'SUPERMX':
            sum = result(operation, num1, num2, num3, num4)
            return render_template('index.html', sum=sum)
        else:
            return render_template('index.html')



if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jun/2020 23:10:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2020 23:28:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2020 23:34:11] "POST /send HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2020 23:34:46] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2020 23:34:53] "POST /index HTTP/1.1" 200 -
